In [1]:
import cv2
import numpy as np

In [46]:
from tqdm import tqdm

In [22]:
# Load the image of the labyrinth
img = cv2.imread("df_maze.png")

In [23]:
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [24]:
# Threshold the image to binary
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

In [25]:
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [26]:
len(binary), len(binary[0])

(504, 504)

In [33]:
# Find the start and goal coordinates
start = None
for y in range(binary.shape[0]):
    for x in range(binary.shape[1]):
        if binary[y, x] == 255:
            if start is None:
                start = (y, x)
                break

In [35]:
end = None
for y in reversed(range(binary.shape[0])):
    for x in reversed(range(binary.shape[1])):
        if binary[y, x] == 255:
            if end is None:
                end = (y, x)
                break

In [50]:
start, end

((5, 5), (498, 498))

In [51]:
goal = end

In [52]:
# Define the possible actions
actions = ["up", "down", "left", "right"]

# Define the Q-table
q_table = np.zeros((binary.shape[0], binary.shape[1], len(actions)))

# Define the learning rate
alpha = 0.8

# Define the discount factor
gamma = 0.95

# Define the exploration rate
epsilon = 0.1

# Define the maximum number of episodes
# episodes = 10000
episodes = 100

In [53]:
# Train the Q-learning algorithm
for episode in tqdm(range(episodes)):
    # Set the initial state
    state = start

    # Loop until the agent reaches the goal
    while (abs(state[0]) < len(binary[0])) and (abs(state[1]) < len(binary[1])):
        # Select the action with the highest Q-value
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(actions)
        else:
            action = actions[np.argmax(q_table[state[0], state[1]])]

        # Take the action and observe the new state and reward
        if action == "up":
            new_state = (state[0] - 1, state[1])
            reward = -1 if binary[new_state[0], new_state[1]] == 0 else 0
        elif action == "down":
            new_state = (state[0] + 1, state[1])
            reward = -1 if binary[new_state[0], new_state[1]] == 0 else 0 
        elif action == "left": 
            new_state = (state[0], state[1] - 1)
            reward = -1 if binary[new_state[0], new_state[1]] == 0 else 0 
        elif action == "right": 
            new_state = (state[0], state[1] + 1)
            reward = -1 if binary[new_state[0], new_state[1]] == 0 else 0
    
        # Update the Q-value
        q_table[state[0], state[1], actions.index(action)] = q_table[state[0], state[1], actions.index(action)] + alpha * (reward + gamma * np.max(q_table[new_state[0], new_state[1]]) - q_table[state[0], state[1], actions.index(action)])

        # Update the state
        state = new_state

        # Check if the goal is reached
        if state == goal:
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [24:52<00:00, 14.92s/it]


In [54]:
state

(-504, 87)

In [55]:
# Print the Q-table
print(q_table)

[[[-1.7216     -2.30528    -2.30528    -2.7731968 ]
  [-2.8051968  -2.9823488  -2.30528    -2.7731968 ]
  [-2.33728    -2.30528    -2.3296     -2.8704768 ]
  ...
  [-1.7216     -1.6896     -1.568      -2.27328   ]
  [-1.7216     -2.8412928  -2.30528    -2.3296    ]
  [-1.7216     -2.30528    -2.397696   -2.397696  ]]

 [[-2.336      -2.397696   -1.8432     -2.35392   ]
  [-2.9024768  -3.0115328  -2.30528    -2.6564608 ]
  [-2.336      -2.30528    -2.18368    -2.9628928 ]
  ...
  [-1.6        -2.30528    -2.30528    -2.27328   ]
  [-2.404096   -2.30528    -2.30528    -2.8899328 ]
  [-2.18368    -2.30528    -2.30528    -2.538752  ]]

 [[-2.428416   -2.30528    -2.3296     -2.884096  ]
  [-3.0143488  -3.14459136 -2.569472   -2.9191168 ]
  [-2.31168    -2.30528    -2.9887488  -2.88896   ]
  ...
  [-2.31168    -2.30528    -2.30528    -2.35392   ]
  [-2.8100608  -2.30528    -2.30528    -1.568     ]
  [-1.6        -2.30528    -2.30528    -2.15168   ]]

 ...

 [[-2.18368    -2.397696   -1.568 